In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
from torch import nn
from torch.utils import data
from d2l import torch as d2l

In [2]:
# 这里使用的ID为0~N-1版本
class Person:
  def __init__(self, id):
    self.id = id
    self.alive = True # 是否存活
    self.offers = []  # 收到的offer
    self.job = -1 # 签约的企业

class Recruit:
  def __init__(self, id, num):
    self.id = id
    self.num = num # 岗位剩余容量
    self.workers = [] # 签约的员工
    
class Pair:
  def __init__(self, per, rec, val):
    self.per = per
    self.rec = rec
    self.pid = per.id
    self.rid = rec.id
    self.val = val
  
  def print(self):
    print(self.pid, self.rid, self.val)

In [3]:
# 属性嵌入，这里的序号是0~N-1
# 用于拼接向量 and 判断行业
a_emb_job = pd.read_csv('./attribute_embedding_position.txt',sep=' ',index_col=0,header=None,na_values=[])
a_emb_per = pd.read_csv('./attribute_embedding_user.txt',sep=' ',index_col=0,header=None,na_values=[])

In [4]:
def is_fw(pid):
  val = fw_list.loc[pid,'是否为废人']
  return val==1

fw_list = pd.read_csv('./fw.txt',sep=' ',index_col=0)

In [5]:
alive = []
for i in a_emb_per.index:
  alive.append(i)
ss = pd.Series(alive)
ss.to_csv('./no_fw.csv')

In [6]:
pca_per_ppd = pd.read_csv('./pca_per.csv',index_col=0,header=None)
pca_job_ppd = pd.read_csv('./pca_job.csv',index_col=0,header=None)

In [7]:
pca_per_myd = pd.read_csv('./pca_per_myd.csv',index_col=0,header=None)
pca_job_myd = pd.read_csv('./pca_job_myd.csv',index_col=0,header=None)

In [8]:
def get_input_ppd(pid,rid):
  per = pca_per_ppd.loc[pid].values
  job = pca_job_myd.loc[rid].values
  all = np.concatenate([per,job],axis=0)
  return all

def get_input_myd(pid,rid):
  per = pca_per_myd.loc[pid].values
  job = pca_job_myd.loc[rid].values
  all = np.concatenate([per,job],axis=0)
  return all

In [9]:
net_ppd = torch.load('./Sigmoid-200.pt')
net_myd = torch.load('./Sigmoid-MYD.pt')

In [10]:
# 硬核条件，这里的序号是0~N-1
hd_job = pd.read_csv('./hardcore_position.csv',index_col=0,header=0,encoding='gbk')
hd_job.index = [i for i in range(hd_job.shape[0])]
hd_per = pd.read_csv('./hardcore_user.csv',index_col=0,header=0,encoding='gbk')
hd_per.index = [i for i in range(hd_per.shape[0])]
hd_job.shape, hd_per.shape

((1575, 4), (10877, 3))

In [11]:
user_job = pd.read_csv("data/user_job.csv")
position_job = pd.read_csv("data/position_job.csv")

user_job_matrix = user_job.values
position_job_matrix = position_job.values

In [12]:
hd_job.head()

,性质,岗位需求量,最大薪资,最小薪资
0,全职,6,20000,12000
1,全职,10,26000,16000
2,全职,100,25000,15000
3,全职,5,8000,5000
4,全职,6,6000,3500


In [13]:
def judge_industry(user,position):
  # return True
  return (user_job_matrix[user]*position_job_matrix[position]).sum()>0

def judge_gongzi(pid,rid): # 0~N-1
  # p_low = hd_per.loc[pid,'预期最小薪资']
  # r_high = hd_job.loc[rid,'最大薪资']
  # return p_low <= r_high
  p2, p1 = hd_per.iloc[pid,:-1]
  r2, r1 = hd_job.iloc[rid,-2:]
  return max(p1,r1)<=min(p2,r2)

def judge_type(pid,rid):
  pt = hd_per.iloc[pid,-1]
  rt = hd_job.iloc[rid,0]
  # print(pt,rt)
  return pt == rt

def judge(pid,rid):
  if judge_buf[pid,rid]>=0:
    return judge_buf[pid,rid]==1
  c = 0
  if judge_industry(pid,rid):
    c+=1
  if judge_gongzi(pid,rid):
    c+=1
  if judge_type(pid,rid):
    c+=1
  # if b1 and b2 and b3:
  if c >= 3:
    judge_buf[pid,rid] = 1
  else:
    judge_buf[pid,rid] = 0
  # print('build buf')
  return judge_buf[pid,rid]==1

judge_buf = np.zeros(shape=(10877,1575),dtype='int')-1
# judge_buf

In [14]:
true_cnt,false_cnt=0,0
lp = tqdm(range(100))
for m in lp:
  i = np.random.randint(10877)
  for n in range(500):
    j = np.random.randint(1575)
    if judge(i,j):
      true_cnt+=1
    else:
      false_cnt+=1
    lp.set_description(f'{true_cnt},{false_cnt},{true_cnt/(true_cnt+false_cnt)}')

1413,13728,0.09332276599960372:  30%|███       | 30/100 [00:25<00:59,  1.18it/s]


KeyboardInterrupt: 

In [15]:
ppd_buf = np.zeros(shape=(10877,1575))-1
myd_buf = np.zeros(shape=(10877,1575))-1
# ppd_buf = {}
# myd_buf = {}
# for i in pca_per_ppd.index:
#   ppd_buf[i] = np.zeros(1575)-1
#   myd_buf[i] = np.zeros(1575)-1

def get_ppd(pid,rid):
  if judge(pid,rid)==False: # 硬性条件不满足为0
    return 0
  elif is_fw(pid):  # 废物为1
    # return 0.1
    ppd = 1 + np.random.random_sample()
  elif ppd_buf[pid][rid]>=0:
    return ppd_buf[pid][rid]
  else:
    ppd = net_ppd(torch.Tensor(get_input_ppd(pid,rid))).item()
    # print(ppd)
    # print(f'ppd({pid},{rid})={ppd}')
  ppd = min(ppd,10)
  ppd = max(ppd,1)/10
  ppd_buf[pid][rid] = ppd
  return ppd

def get_myd(pid,rid):
  if judge(pid,rid)==False: # 硬性条件不满足为0
    return 0
  elif is_fw(pid):  # 废物为1
    # return 0.1
    myd = 1 + np.random.random_sample()
  elif myd_buf[pid][rid]>=0:
    return myd_buf[pid][rid]
  else:
    myd = net_myd(torch.Tensor(get_input_myd(pid,rid))).item()
    # print(myd)
    # print(f'ppd({pid},{rid})={ppd}')
  myd = min(myd,10)
  myd = max(myd,1)/10
  myd_buf[pid][rid] = myd
  return myd

In [16]:
get_myd(1,4)

0

In [18]:
reid_per = {}
reid_rec = {}

dict_per = pd.read_excel('./user.xlsx')
for i in dict_per.index:
  reid_per[i]=dict_per.loc[i,'求职者 ID']

dict_rec = pd.read_excel('./position.xlsx')
for i in dict_rec.index:
  reid_rec[i]=dict_rec.loc[i,'招聘信息 ID']


In [19]:
persons = []
recruits = []

In [20]:
for i in tqdm(range(hd_per.shape[0])):
# for i in tqdm(range(5000)):
  # id = hd_per.loc[i, '序号']
  tmp = Person(i)
  persons.append(tmp)
  # print(id)

100%|██████████| 10877/10877 [00:00<00:00, 1088193.98it/s]


In [21]:
for i in tqdm(range(hd_job.shape[0])):
# for i in tqdm(range(50)):
  # id = hd_rec.loc[i+1, '序号']
  num = hd_job.loc[i, '岗位需求量']
  tmp = Recruit(i, num)
  recruits.append(tmp)
  # print(id)

100%|██████████| 1575/1575 [00:00<00:00, 98390.38it/s]


In [22]:
def get_cnt_rec():
  sum = 0
  for i in recruits:
    sum += i.num
  return sum

old_cnt_rec = get_cnt_rec()
old_cnt_rec

5520

In [41]:
# epoch = 0
# total_pair = 0
# cnt_rec = get_cnt_rec()

while True:
  epoch += 1
  # 招聘回合
  print(f'epoch {epoch}:')
  
  np.random.shuffle(persons) # 随机优化
  
  # for per in persons:
  #   per.offers.clear()
  
  for rec in tqdm(recruits):  # 对于每个岗位
    # 发放offer
    if rec.num <= 0:
      continue
    offer_list = [] # 临时数组：<per, rec, val>
    # 计算与每个求职者的匹配度
    for per in persons:
      if per.alive==False:
        continue
      val = get_ppd(per.id, rec.id)
      if val<=0:
        continue
      pair = Pair(per, rec, val)
      offer_list.append(pair)
    # 就地排序 by val
    if len(offer_list)<=0:
      continue
    
    # offer_list.sort(reverse=True, key=lambda x:x.val)
    
    # for of in offer_list:
    #   if get_ppd(of.pid,of.rid)==0:
    #     print(f'1.ppd={get_ppd(of.pid,of.rid)}, val={of.val}')
        
    # if len(offer_list)>3*rec.num:
    #   offer_list = offer_list[:3*rec.num] # 取TopK
    
    for pair in offer_list:
      if pair.val>0:
        pair.per.offers.append(pair) # 发放offer
  
  # 所有岗位offer发放完毕
    
  # 求职者签约
  for per in persons: # 对于每个求职者
    if per.alive == False:
      continue
    # offer_list2 = [] # 临时数组：Pair
    for of in per.offers:  # 对于每一份offer
      # rec = recruits[rid]
      val = get_myd(of.pid, of.rid)
      of.val = val
    #   pair = Pair(per, rec, val)
    #   offer_list2.append(pair)
    # if len(offer_list2)==0:  # 当前求职者无offer
    #   continue
    
    # offer_list2.sort(reverse=True, key=lambda x:x.val) # 取最高满意度
    per.offers.sort(reverse=True, key=lambda x:x.val)
    # 目标岗位
    for of in per.offers:
      # dest_rec = recruits[k.rid]
      if of.rec.num > 0: # 若未招满
        of.rec.num -= 1
        per.job = of.rid
        per.alive = False
        # per.offers.clear()
        of.rec.workers.append(per.id)
        # if of.pid!=per.id:
        #   print('ID ERR!')
        # 检测非法值
        # if get_myd(per.id,of.rid)==0:
        #   print(f'2.<{per.id},{of.rid}>:myd={get_myd(per.id,of.rid)},ppd={get_ppd(per.id,of.rid)}')
        break
    
    per.offers.clear()
      
  # 删除冗余
  
  new_cnt_rec = get_cnt_rec()
  increase = cnt_rec - new_cnt_rec
  cnt_rec = new_cnt_rec
  total_pair += increase
  # print(net1_cnt, net2_cnt)
  print(f'increase {increase}, total {total_pair}\n')
  
  # break
  
  if increase <= 0:
    break

epoch 3:


100%|██████████| 1575/1575 [00:29<00:00, 52.66it/s] 

increase 0, total 818



In [45]:
# 求职者offer情况
for i in persons:
  if i.job!=-1:
    print(f'{i.id},{i.alive},offers:{len(i.offers)}')

30,False,offers:0
3554,False,offers:0
6629,False,offers:0
652,False,offers:0
1189,False,offers:0
118,False,offers:0
6850,False,offers:0
7763,False,offers:0
2768,False,offers:0
8819,False,offers:0
2838,False,offers:0
10729,False,offers:0
244,False,offers:0
2916,False,offers:0
5808,False,offers:0
5759,False,offers:0
8367,False,offers:0
198,False,offers:0
337,False,offers:0
9793,False,offers:0
8417,False,offers:0
1485,False,offers:0
10227,False,offers:0
3523,False,offers:0
1868,False,offers:0
1009,False,offers:0
787,False,offers:0
6797,False,offers:0
181,False,offers:0
45,False,offers:0
2932,False,offers:0
169,False,offers:0
2366,False,offers:0
484,False,offers:0
138,False,offers:0
4600,False,offers:0
7242,False,offers:0
4992,False,offers:0
10550,False,offers:0
3609,False,offers:0
203,False,offers:0
102,False,offers:0
10096,False,offers:0
4755,False,offers:0
8976,False,offers:0
10728,False,offers:0
6111,False,offers:0
3058,False,offers:0
57,False,offers:0
168,False,offers:0
4383,False,off

In [44]:
st = set()
cnt_pair = 0
with open(f'./res/result4_{total_pair}.csv','w') as f:
  f.write('招聘信息 ID,求职者 ID,岗位匹配度,求职者满意度\n')
  for rec in recruits: # 对于每个岗位
  #print(f'岗位{rec.id}, 招到{len(rec.workers)}, 还差{rec.num}' ,end='\t: ')
  #if len(rec.workers)>0:  # 若非空
    for pid in rec.workers: # 对于每个签约求职者
      # if get_ppd(pid,rec.id)!=0:
      #   continue
      print(f'<{pid},{get_ppd(pid,rec.id)},{get_myd(pid,rec.id)}>', end=' ')
      f.write(f'{reid_rec[rec.id]},{reid_per[pid]},{get_ppd(pid,rec.id)},{get_myd(pid,rec.id)}\n')
      cnt_pair += 1
      st.add(pid)
    if len(rec.workers)>0:
      print('')
    
print(cnt_pair, len(st))

<1904,0.13942026285981646,0.11780830590331523> <9263,0.13860040280860586,0.14404683564017604> <9538,0.12869641089217676,0.17504525187514042> <1054,0.11837507228223546,0.10591180202193935> <1991,0.19644233528157912,0.11726374303823109> 
<4373,0.15283544442227753,0.1944573149103222> <6387,0.156471485528423,0.18614320873297868> <267,0.14318141775290255,0.11483102296706194> <7795,0.16706559988168884,0.17748692518693382> <10659,0.1912330314348647,0.1862537718656546> <6899,0.1722111025290484,0.17344951114682283> 
<1983,0.13681900633026509,0.12451811420203171> <9535,0.1401740908940136,0.15213342428405469> <2879,0.1794989906967736,0.18123597921185447> <10486,0.17849819488163896,0.15437833836024278> <5212,0.17047029854160126,0.15115306687658872> <10758,0.1135297135513103,0.18027142643459065> <2959,0.13000606017104852,0.1610208565052075> <4725,0.10291672532987386,0.11739723518099247> <9562,0.1096286327686121,0.16642439921118268> <1359,0.1701443231761237,0.12495066296414273> 
<8347,0.197746617682

In [22]:
df_4 = pd.read_csv('./res/result4_818.csv')
df_4.head()

,招聘信息 ID,求职者 ID,岗位匹配度,求职者满意度
0,1648165203084440064,7535404855857959936,0.153096,0.168230
1,1648165203084440064,7538323157286569984,0.164802,0.138903
2,1648165203084440064,7538753074922970112,0.162694,0.140361
3,1648165203084440064,7537851110315969536,0.182889,0.188169
4,1648165203084440064,7535276054083720192,0.143152,0.124577


In [23]:
df_4 = df_4.sort_values(by=['招聘信息 ID','岗位匹配度'],ascending=[True,False])
df_4.to_csv('./res/result4_last.csv',index=0)
df_4.head()

,招聘信息 ID,求职者 ID,岗位匹配度,求职者满意度
476,1471751694089060096,1469186255756590080,0.165690,0.110064
477,1471751694089060096,1480370431885179904,0.121189,0.173729
817,1471754691439360000,1630818546172950016,0.746682,1.000000
816,1478265482669850112,1469233133072280064,0.885073,0.962209
813,1478265482669850112,1470960841070340096,0.881648,1.000000


In [46]:
result12 = pd.read_csv('./res/result1-2.csv')
result12.head()
result12.values[0,9]
# result12.to_csv('./res/result1-2_new.csv',index=0)

'本科毕业，具有多年的比赛经验，曾获数据挖掘一等奖，数据分析一等奖，数学竞赛二等奖，数学建模一等奖，练习时长两年半。\n'

In [49]:
# for i in result12.index:
#   print(i)
  # result12.loc[i,'自我评价'] = result12.values[i,9].replace('\n',' ')
  # print(result12.loc[i,'自我评价'])
for index,row in tqdm(result12.iterrows()):
  print(row[9])

3450it [00:00, 17343.32it/s]

本科毕业，具有多年的比赛经验，曾获数据挖掘一等奖，数据分析一等奖，数学竞赛二等奖，数学建模一等奖，练习时长两年半。 
法律改革
再看一眼就会爆炸
awdaw
我TM就是一个傻杯
精通...
善于交际，认真负责
泰迪杯挑战赛二等奖
熟悉Python、SPSS、JavaScript、MATLAB、C++等多门编程语言和数据分析工具，掌握数据挖掘、数学建模、机器学习、小程序开发等技能，兴趣广泛，学习能力强。
awdawd
熟练使用python、c++等工具
非常厉害，并且会鸡你太美
掌握python、C语言的基础编程以及常用操作； 熟 悉numpy、pandas、 tensorFlow架构； 熟悉 MySQL 数据库，熟练编写 SQL 语句； 掌握 机器学习的基础算法；
熟悉VR游戏制作，UE5虚拟引擎，精通C++、Python、matlab等编程语言。七年编程经验。熟悉pytorch、TensorFlow等深度学习框架。主要熟悉的领域有图像、NLP、强化学习，模拟飞行器开发、游戏开发、网页爬虫、网站逆向分析等。
熟练掌握python，Java编程技术，有前端开发经验
爱好
无
啥都会
啥都会
啥都会
Python 数据分析
无
熟练操作python 、爬虫、sql语句、excel和powerbi等软件
本人是一名退伍军人,保留军人本色,为人平和,对待工作严谨,具备敏锐的观察力,对数字敏感, 熟悉 Linux,熟练运用 SQL、excel、PowerBi、SPSS、Python 等软件独立完成数据收集、整理、清洗、分析及可视化、挖掘,同时具备大数据相关组件构成的项目经验。
担任过班干、技术部部长、项目负责人因此具有一定组织力， 技术方面：荣获python编程国赛三等奖及人社部python程序员三级职业技能证书，科大讯飞1+X中级数据分析师。
熟悉MySQL和Python 
熟练使用 Python 语言，了解基础机器学习和深度学习算法和模型，多次参加数据类比赛并获奖，有一定的数据分析经验和能力。对数据库基础操作基本掌握。 
富有团队合作精神，善于与他人沟通交流； 处事态度细心谨慎，责任心强。具有扎实的数据科学专业基础知识，掌握常见的数据挖掘和分析方法。多次参加大学生学科竞赛的经验，熟悉数据分析/数据挖掘模型的建立，能够独立完成、撰写数据分析报告。
富有团队合作精神，善于与他

7244it [00:00, 18344.73it/s]

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


10877it [00:00, 18218.82it/s]

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [46]:
obj = cnt_pair/old_cnt_rec
obj

0.14818840579710144

In [50]:
# 调用实例，使用时可以把函数里的print关掉
# 仅第一次调用神经网络，即建立缓存时会print
a,b = 317,46
get_ppd(a,b),get_myd(a,b)

(0, 0)

In [47]:
# 以下为导出文件

In [47]:
user_map = pd.read_csv("map/user_map.csv")
position_map = pd.read_csv("map/position_map.csv")

In [48]:
ppd_list = []
for p_index,p_row in tqdm(position_map.iterrows()):
    cur = []
    for u_index,u_row in user_map.iterrows():
        ppd = get_ppd(u_index,p_index)
        if ppd>0:
            cur.append((p_row["招聘信息 ID"],u_row["求职者 ID"],ppd))
            
    # cur.sort(key=lambda x:x[2],reverse=True)
#             print(cur)
    ppd_list += cur
#     print(ppd_list)

1575it [19:01,  1.38it/s]


In [49]:
ppd_df = pd.DataFrame(data=ppd_list,columns=["招聘信息 ID","求职者 ID","岗位匹配度"])
ppd_df.sort_values(by=['招聘信息 ID','岗位匹配度'], inplace=True, ascending=[True,False])
ppd_df.to_csv("res/ppd_last.csv",index=0)

In [50]:
ppd_list = []
myd_list = []
for u_index,u_row in tqdm(user_map.iterrows()):
    cur = []
    for p_index,p_row in position_map.iterrows():
        myd = get_myd(u_index,p_index)
        if myd>0:
            cur.append((u_row["求职者 ID"],p_row["招聘信息 ID"],myd))
            
    # cur.sort(key=lambda x:x[2],reverse=True)
    myd_list += cur

10877it [14:38, 12.38it/s]


In [51]:
myd_df = pd.DataFrame(data=myd_list,columns=["求职者 ID","招聘信息 ID","求职者满意度"])
myd_df = myd_df.merge(position_map,on="招聘信息 ID",how="left")
myd_df = myd_df[["求职者 ID","招聘信息 ID","企业名称","求职者满意度"]]
myd_df.sort_values(by=['求职者 ID','求职者满意度'], inplace=True, ascending=[True,False])
myd_df.rename(columns={'企业名称':'公司名称'},inplace = True)
# myd_df.sort_values(by=['求职者满意度'], inplace=True)
myd_df.to_csv("res/myd_last.csv",index=0)

In [85]:
# myd_df.to_csv("res/myd-ansi.csv",index=0,encoding='ansi')

In [137]:
myd_df.head()

,求职者 ID,招聘信息 ID,公司名称,求职者满意度
27677,1461512488951611392,1482192956965388300,威尔森,0.199584
27556,1461512488951611392,1482196148411301894,北京数起科技有限公司,0.199417
27584,1461512488951611392,1482195175148224537,广东中思拓大数据研究院有限公司,0.198925
27490,1461512488951611392,1629824814967554048,太普软件,0.198744
27709,1461512488951611392,1482192126010851334,精鼎医药研究开发(上海)有限公司,0.198173
